<div style="background-color: #f5f5f5; padding: 15px; color: black; width: 80%;">✍ Библиотека Pandas предоставляет большое количество возможностей для преобразований данных, однако иногда необходимо совершать более сложные манипуляции над столбцами. Например, из столбцов, содержащих в себе некоторый текст, необходимо специальным образом извлечь определённые слова, даты или числа.</div>

<div style="background-color: #e0fff3; padding: 15px; color: black; width: 80%;">→ Для таких случаев Pandas не имеет специальных методов, однако позволяет расширить свою функциональность за счёт использования пользовательских функций. </div>

Мы можем написать некоторую функцию, которая принимает на вход один элемент столбца, каким-то образом его обрабатывает и возвращает результат, после чего применить эту функцию к каждому элементу в столбце с помощью специального метода apply(). В результате применения этой функции будет возвращён объект Series, элементы которого будут представлять результат работы этой функции.

In [105]:
import pandas as pd

In [106]:
melb_data = pd.read_csv("../Module4/data/melb_data.csv")
melb_df = melb_data.copy()

---
Рассмотрим пример. В наших данных есть столбец с адресами объектов недвижимости. Проблема этого столбца в том, что в нём слишком большое количество уникальных значений: почти на каждый объект недвижимости в таблице приходится свой уникальный адрес. Убедимся в этом, вычислив количество уникальных значений в столбце с помощью метода nunique():

In [107]:
print(melb_df["Address"].nunique())

13378


Если мы прогнозируем цену объекта, то такое большое количество возможных категорий может плохо сказаться на модели, которую мы бы хотели в дальнейшем построить на наших данных. Говорят, что такой признак, скорее всего, не имеет <b>статистической значимости,</b> потому что не позволяет разделить данные на группы, которые можно сравнить по целевому признаку.

<div style="background-color: #ffdcd4;color: black;border: 3px solid black; padding: 15px; margin-right: 500px; width: 80%;">Из-за таких признаков зависимость между целевым признаком, который мы хотим предсказать, и признаками, на основе которых мы делаем предсказание, становится очень сложной. При этом точность моделирования при учёте такого признака может не повыситься, а даже снизиться, а производительность однозначно резко упадёт.</div>

Обычно подобные признаки удаляют, однако можно поступить умнее: давайте извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар). Для этого сначала внимательнее посмотрим на структуру адреса, выберем несколько строк столбца Address:

In [108]:
print(melb_df["Address"].loc[177])
print(melb_df["Address"].loc[1812])
print(melb_df["Address"].loc[9001])
# 2/119 Railway St N
# 9/400 Dandenong Rd
# 172 Danks St

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


Итак, адрес строится следующим образом: сначала указывается номер дома и корпус, после указывается название улицы, а в конце — подтип улицы, но в некоторых случаях к подтипу добавляется географическая отметка (N — север, S — юг и т. д.), она нам не нужна . Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:

In [109]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
    # Создаём список географических пометок exclude_list.
    exclude_list = ["N", "S", "W", "E"]
    # Метод split() разбивает строку на слова по пробелу.
    # В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(" ")
    # Обрезаем список, оставляя в нём только последний элемент,
    # потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
    # Делаем проверку на то, что полученный подтип является географической пометкой.
    # Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
        # Если переменная street_type является географической пометкой,
        # переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
    # Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

Теперь применим эту функцию к столбцу c адресом. Для этого передадим функцию get_street_type в аргумент метода столбца apply(). В результате получим объект Series, который положим в переменную street_types:

In [110]:
street_types = melb_df["Address"].apply(get_street_type)
display(street_types)

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

Обратите внимание, что функция пишется для одного элемента столбца, а метод apply() применяется к каждому его элементу. Используемая функция обязательно должна иметь возвращаемое значение.

Итак, мы смогли выделить подтип улицы. Посмотрим, сколько уникальных значений у нас получилось:

In [111]:
print(street_types.nunique())

56


У нас есть 56 уникальных значений. Однако наш результат можно улучшить. Давайте для начала посмотрим на частоту каждого подтипа улицы с помощью метода value_counts:

In [112]:
display(street_types.value_counts())

Address
St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Gdns            4
Grn             4
Fairway         4
Mews            4
Crossway        3
Righi           3
Victoria        2
Ridge           2
Crofts          2
Esp             2
Glade           1
Gra             1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Athol           1
Summit          1
Grand           1
Res             1
Nook            1
Eyrie           1
Dell            1
East            1
Loop            1
Grange          1
Terrace         1
Cove            1
Qy              1
Co

Из данного вывода можно увидеть, что есть группа наиболее популярных подтипов улиц, а дальше частота подтипов быстро падает.

В таком случае давайте применим очень распространённый метод уменьшения количества уникальных категорий — выделим n подтипов, которые встречаются чаще всего, а остальные обозначим как 'other' (другие).

Для этого к результату метода value_counts применим метод nlargest(), который возвращает n наибольших значений из Series. Зададим n=10, т. е. мы хотим отобрать десять наиболее популярных подтипов. Извлечём их названия с помощью атрибута index, а результат занесём в переменную popular_stypes:

In [113]:
popular_stypes = street_types.value_counts().nlargest(10).index
popular_stypes
# Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object', name='Address')

<div style="background-color: #e0ffd1;color: black;border: 3px solid black; padding: 15px; margin-right: 500px; width: 80%;">Попробуйте применить использованные методы последовательно и выводить результат, чтобы проследить за логикой преобразования.</div>

Теперь, когда у нас есть список наиболее популярных подтипов улиц, введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, и, если это так, lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'. Наконец, применим такую функцию к Series street_types, полученной ранее, а результат определим в новый столбец таблицы StreetType:

In [114]:
melb_df["StreetType"] = street_types.apply(
    lambda x: x if x in popular_stypes else "other"
)
display(melb_df["StreetType"])

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

Посмотрим на результирующее число уникальных подтипов:

In [115]:
melb_df.StreetType.nunique()

11

Теперь, у нас нет потребности хранить признак Address, так как, если конкретное местоположение объекта всё же и влияет на его стоимость, то оно определяется столбцами Longitude и Lattitude. Удалим его из нашей таблицы:

In [116]:
melb_df = melb_df.drop("Address", axis=1)

Tаким образом, с помощью написания собственных функций и их комбинирования с методом apply() из библиотеки Pandas мы смогли извлечь информацию из признака с адресом и заменить на признак подтипа улицы.


<div style="background-color: #e0ffd1;color: black;border: 3px solid black; padding: 15px; margin-right: 500px; width: 80%;">
Примечание. Внимательный читатель наверняка обратит внимание на то, что мы допустили небольшую ошибку!

Если присмотреться, то в списке подтипов улиц street_types можно заметить подтипы, которые именуются различным образом, но при этом обозначают одинаковые вещи. Например, подтипы Av и Avenue, Bvd и Boulevard, Pde и Parade. Мы упустили данный момент, хотя в реальных задачах стоит обращать пристальное внимание на результаты преобразований и исправлять неточности в данных.

Такие ошибки в данных (обозначение идентичных категорий различными именами) являются одним из видов «грязных» данных.

Порой отследить такие неточности бывает очень сложно, а при наличии большого количества категорий (например, более ста) — практически невозможно.

Мы предлагаем вам самостоятельно разобраться с этой ошибкой: попробуйте написать функцию-преобразование (lambda-функцию-преобразование), которая возвращала бы вместо значений Avenue, Boulevard и Parade их топографические сокращения, и примените её к данным о подтипах улиц.

→ Обратите внимание, что данное преобразование необходимо применить до сокращения количества уникальных категорий.
</div>

In [117]:
melb_df = melb_data.copy()

In [118]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
    # Создаём список географических пометок exclude_list.
    exclude_list = ["N", "S", "W", "E"]
    # Метод split() разбивает строку на слова по пробелу.
    # В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(" ")
    # Обрезаем список, оставляя в нём только последний элемент,
    # потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
    # Делаем проверку на то, что полученный подтип является географической пометкой.
    # Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
        # Если переменная street_type является географической пометкой,
        # переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
    # Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

In [119]:
street_types = melb_df["Address"].apply(get_street_type)
street_types.value_counts()

Address
St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Gdns            4
Grn             4
Fairway         4
Mews            4
Crossway        3
Righi           3
Victoria        2
Ridge           2
Crofts          2
Esp             2
Glade           1
Gra             1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Athol           1
Summit          1
Grand           1
Res             1
Nook            1
Eyrie           1
Dell            1
East            1
Loop            1
Grange          1
Terrace         1
Cove            1
Qy              1
Co

In [120]:
repeats = {"Avenue": "Av", "Boulevard": "Bvd", "Parade": "Pde"}
street_types = street_types.apply(
    lambda x: x if x not in repeats.keys() else repeats[x]
)
street_types.value_counts()

Address
St           8012
Rd           2825
Ct            612
Dr            447
Av            361
Gr            311
Pde           226
Pl            169
Cr            152
Cl            100
La             67
Bvd            66
Tce            47
Wy             40
Cct            25
Hwy            24
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Mews            4
Grn             4
Fairway         4
Gdns            4
Righi           3
Crossway        3
Esp             2
Victoria        2
Ridge           2
Crofts          2
Grand           1
Summit          1
Hts             1
Athol           1
Highway         1
Outlook         1
Woodland        1
Ave             1
Gra             1
Terrace         1
Eyrie           1
Dell            1
East            1
Loop            1
Nook            1
Glade           1
Qy              1
Cove            1
Res             1
Grange          1
Corso           1
Name: count, dtype: int64

In [121]:
popular_stypes = street_types.value_counts().nlargest(10)
popular_stypes

Address
St     8012
Rd     2825
Ct      612
Dr      447
Av      361
Gr      311
Pde     226
Pl      169
Cr      152
Cl      100
Name: count, dtype: int64

In [122]:
street_types = street_types.apply(lambda x: x if x in popular_stypes else "other")
street_types.value_counts()

Address
St       8012
Rd       2825
Ct        612
Dr        447
other     365
Av        361
Gr        311
Pde       226
Pl        169
Cr        152
Cl        100
Name: count, dtype: int64

In [123]:
melb_df["StreetType"] = street_types

Резюмируя, поделимся общими рекомендациями по уменьшению числа уникальных значений в признаке, который описывается категориями:

<div style="border: 1px solid white; padding: 5px; margin-right: auto;  width: 80%;"> 
<div style="color: white;background-color: black;">1</div>
 Определите (хотя бы на глаз) соотношение числа уникальных категорий интересующего вас признака к общему числу объектов в таблице. Если это соотношение превышает значение 30 %, то это уже повод задуматься над уменьшением числа категорий и перейти к шагу 2.
 </div>

<div style="border: 1px solid white; padding: 5px; margin-right: auto;  width: 80%;"> 
<div style="color: white;background-color: black;">2</div>
Если ваш признак уникален для каждого объекта, например адрес, имя или название, то такой признак, скорее всего, не имеет статистической значимости. От таких признаков чаще всего избавляются. Однако можно попробовать выделить из этого признака какие-то общие черты, например, как мы это сделали с подтипами улиц. Такой же трюк можно произвести, например, с названиями компаний, в которых может быть скрыт признак типа организации (из строки «ООО Три Слепые Мыши» можно извлечь ООО — общество с ограниченной ответственностью).

Далее переходите к шагу 3.
 </div>

<div style="border: 1px solid white; padding: 5px; margin-right: auto;  width: 80%;"> 
<div style="color: white;background-color: black;">3</div>
Если даже после преобразования число уникальных категорий всё ещё велико, можно попробовать с помощью метода value_counts() оценить, есть ли в данных категории, которые употребляются гораздо реже, чем остальные. Если такие категории присутствуют, переходите к шагу 4.
 </div>

<div style="border: 1px solid white; padding: 5px; margin-right: auto;  width: 80%;"> 
<div style="color: white;background-color: black;">4</div>
Можно подобрать число  популярных категорий таким образом, чтобы эти категории покрывали большую часть ваших данных.

Когда вы выбрали оптимальное число, переходите к шагу 5.
 </div>

<div style="border: 1px solid white; padding: 5px; margin-right: auto;  width: 80%;"> 
<div style="color: white;background-color: black;">5</div>
Наконец, можно совершить преобразование, обозначив категории, не попавшие в число популярных, как «другие».
 </div>

<div style="background-color: #f5f5f5; padding: 15px; color: black; width: 80%;">→ Такая методика является очень популярной и хорошо показывает себя на практике. Однако не нужно ей злоупотреблять: применяя эту методику ко всем столбцам подряд, вы рискуете потерять немалую долю полезной информации из ваших данных. Внимательно изучите интересующий вас признак, прежде чем преобразовывать его.</div>

###  Задание 4.1

Загляните в документацию по методу apply() и определите, с помощью какого параметра данного метода можно передавать другие аргументы в вызываемую функцию.
- new_args
- args
- axis
- result_type 

<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
- args
</code>
</details>

###  Задание 4.2

Ранее, в задании 3.3, мы создали признак WeekdaySale в таблице melb_df — день недели продажи. Из полученных в задании результатов можно сделать вывод, что объекты недвижимости в Мельбурне продаются преимущественно по выходным (суббота и воскресенье).

Напишите функцию get_weekend(weekday), которая принимает на вход элемент столбца WeekdaySale и возвращает 1, если день является выходным, и 0 — в противном случае, и создайте столбец Weekend в таблице melb_df с помощью неё.

Примените эту функцию к столбцу и вычислите среднюю цену объекта недвижимости, проданного в выходные дни. Результат округлите до целых.

<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
Подсказка (1 из 3): Создайте функцию get_weekend. На вход данной функции подайте элемент столбца WeekdaySale.
</code>
</details>

In [131]:
melb_df["Date"] = pd.to_datetime(melb_df["Date"], dayfirst=True)
display(melb_df["Date"])

0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]

In [135]:
melb_df["WeekdaySale"] = melb_df.Date.dt.weekday
melb_df.WeekdaySale

0        5
1        3
2        5
3        5
4        5
        ..
13575    5
13576    5
13577    5
13578    5
13579    5
Name: Date, Length: 13580, dtype: int32

In [137]:
def get_weekend(weekday):
    if weekday in [5, 6]:
        return 1
    else:
        return 0

In [139]:
melb_df["Weekend"] = melb_df.WeekdaySale.apply(get_weekend)
melb_df.Weekend

0        1
1        0
2        1
3        1
4        1
        ..
13575    1
13576    1
13577    1
13578    1
13579    1
Name: Weekend, Length: 13580, dtype: int64

In [141]:
melb_df[melb_df.Weekend == 1]["Price"].mean()

1081198.6406956792

###  Задание 4.3

Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df следующим образом: оставьте в столбце только 49 самых популярных компаний, а остальные обозначьте как 'other'.

Найдите, во сколько раз минимальная цена объектов недвижимости, проданных компанией 'Nelson', больше минимальной цены объектов, проданных компаниями, обозначенными как 'other'. Ответ округлите до десятых.

<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
 Подсказка (1 из 1): Метод решения задачи аналогичен методу поиска самых популярных улиц, описанному в данном юните. 1.3
</code>
</details>

In [142]:
melb_df.SellerG.value_counts()

SellerG
Nelson           1565
Jellis           1316
hockingstuart    1167
Barry            1011
Ray               701
                 ... 
Prowse              1
Luxe                1
Zahn                1
Homes               1
Point               1
Name: count, Length: 268, dtype: int64

In [146]:
popular_sellers = melb_df["SellerG"].value_counts().nlargest(49)
melb_df["SellerG"] = melb_df["SellerG"].apply(
    lambda x: x if x in popular_sellers else "other"
)

In [148]:
melb_df[melb_df["SellerG"] == "Nelson"]["Price"].min() / melb_df[
    melb_df["SellerG"] == "other"
]["Price"].min()

1.297709923664122

### Задание 4.4

Представьте, что вы занимаетесь подготовкой данных о вакансиях с платформы hh.ru. В вашем распоряжении имеется таблица, в которой с помощью парсинга собраны резюме кандидатов. В этой таблице есть текстовый столбец «Опыт работы». Пример такого столбца представлен ниже в виде объекта Series. Структура текста в столбце фиксирована и не может измениться.

Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. Функция должна возвращать опыт работы в месяцах. Не забудьте привести результат к целому числу.

Примечание. Обратите внимание, что опыт работы может выражаться только в годах или только в месяцах. Учтите это при построении своей функции.

При проверке мы будем применять вашу функцию к разным Series с помощью метода apply().

Пример результата работы функции get_experience:<br>
<img src="../static//img/847_1.png"> <img src="../static//img/847_2.png">


In [149]:
def get_experience(arg):
    text_list = arg.split()
    if len(text_list) > 4:
        years = int(text_list[2])
        months = int(text_list[-2])
        return years * 12 + months
    else:
        if text_list[-1].startswith("мес"):
            return int(text_list[-2])
        else:
            years = int(text_list[2])
            return years * 12